In [1]:
import pandas as pd
#from urllib import request
from urllib.request import urlopen,Request
from bs4 import BeautifulSoup

In [2]:
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
def getscrape(url,coding):
    response = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
    raw = urlopen(response).read()
    raw= raw.decode(coding)
    raw = BeautifulSoup(raw, 'html.parser')
    return raw

In [3]:
def getelement(raw,element,attri1, attri2):
    setraw = ""
    if(attri1 == None):
        setraw = str(raw.find_all(element))
    else:
        setraw = str(raw.find_all(element,attrs={attri1:attri2}))
    return setraw


In [4]:
def martiniqueCreole():
    martinique = getscrape("https://www.potomitan.info/dictionnaire/francais.php",'ISO-8859-1')
    martinique = getelement(martinique,'p',None,None)
    martinique = martinique.split("<br/>")
    martinique.pop(0)
    Clean = []
    for x in range(0,len(martinique)):
        if "</p>, <p>" in martinique[x]:
            l = martinique[x].split("</p>, <p>")
            martinique[x] = l[0]
            Clean.append(l[1])
    Clean = Clean[0:21]
    martinique = martinique[0:621]

    for x in Clean:
        martinique.append(x)
    return martinique

In [5]:
martinique = martiniqueCreole()
french = []
creole = []
name = []
for x in martinique:
    l = x.split(':')
    if x != '\n':
        french.append(l[0].strip('\n                  '))
        creole.append(l[1])
        name.append("Martinique")
df = pd.DataFrame()
df['Acrolect'] = french
df['Creole_word'] = creole
df['Creole_name'] = name
#df.to_csv("Excelmartinique.csv")

In [39]:
defandWord = pd.DataFrame(columns=['Creole_word','Definition'])
def stLucia():
    word = []
    defi = []
    links = []
    for x in alphabet:
        a = 'https://www.webonary.org/kweyol/browse/kweyol-english/?letter='+x+'&key=acf&lang=fr'
        Lucia =  getscrape(a,'utf8')
        l = str(getelement(Lucia,'div','id','wp_page_numbers'))
        if len(l) == 0:
            links.append(a)
        else:
            temp = []
            m = l.replace("</a></li>",'').replace("<a href=",'').split("<li>")
            for y in m:
                if "page_info" not in y and "</div>" not in y:
                    temp.append(y)
            r = temp.pop(-1)
            r = r.split("amp;")
            if r[-1][-3:-1] != "[":
                c = int(r[-1][-3:-1].replace('>',''))
                for n in range(1,(c+1)):
                    i = 'https://www.webonary.org/kweyol/browse/kweyol-english/'+r[0][1:]+r[1]+r[2]+"pagenr="+str(n)+"&"+r[3][:-1]
                    links.append(i)
    
    for x in links:
        luciaW = getscrape(x,'utf8')
        w = str(getelement(luciaW,'div','class','post')).split("<html>")
        for y in w:
            if "definitionorgloss" in y:
                m = y.split("<span")
                l = []
                l2 = []
                v = ""
                dict1 = {}
                for p in m:
                    if "lang=\"acf\"><a" in p:
                        z = p.split("?lang=fr\">")
                        z.pop(0)
                        z = z[0].split("</a>")
                        l.append(z[0])
                    if "lang=\"en\"" in p:
                        z = p.split("</")
                        if len(z) == 3:
                            z = z[0].replace(" lang=\"en\">",'')
                            if z != "cond. var." and z != "syn" and z != " ":
                                l2.append(z)
                defi.append(','.join(l2))
                word.append(','.join(l))
                
    defandWord['Creole_word'] = word
    defandWord['Definition'] = defi


    #print(defandWord)

In [41]:
stLucia()
#this csv is not perfect have to make correction in excel sheets.
#defandWord.to_csv("stLucia.csv")